# Endpoint Tester

Sends message to deployed endpoint and prints returned prediction.

In [12]:
msg = "hey i'm david glad to help you what's up galaxy s8 checkout ringtone details answer view more contacts k thx bye"

In [20]:
import tensorflow as tf
import os
from sagemaker.tensorflow.predictor import tf_serializer, tf_deserializer
from sagemaker.predictor import RealTimePredictor
from tensorflow.python.saved_model.signature_constants import DEFAULT_SERVING_SIGNATURE_DEF_KEY
from tensorflow_serving.apis import classification_pb2
from utils.pre_processing import preprocess_data

ENDPOINT_NAME = 'sagemaker-tensorflow-py2-cpu-2018-04-11-19-07-13-402'
QUESTION = 1


def create_feature(v):
    return tf.train.Feature(int64_list=tf.train.Int64List(value=v))

def build_request(name, features, signature_name=DEFAULT_SERVING_SIGNATURE_DEF_KEY):
    request = classification_pb2.ClassificationRequest()
    request.model_spec.name = name
    request.model_spec.signature_name = signature_name

    example = tf.train.Example(features=tf.train.Features(feature=features))
    request.input.example_list.examples.extend([example])
    
    return request

def transform(msg):
    msg_transform = preprocess_data([msg])
    vocab_file = os.path.join("data", "vocab_question_{}".format(QUESTION))
    vocab_processor = tf.contrib.learn.preprocessing.VocabularyProcessor.restore(vocab_file)
    msg_transform = vocab_processor.transform(msg_transform)
    return list(msg_transform)[0]

In [21]:
predictor = RealTimePredictor(endpoint=ENDPOINT_NAME,
                              serializer=tf_serializer,
                              content_type='application/octet-stream')
features = {'words': create_feature(transform(msg))}
request = build_request("generic_model", features)

In [22]:
print(predictor.predict(request))

result {
  classifications {
    classes {
      label: "0"
      score: 0.39432576298713684
    }
    classes {
      label: "1"
      score: 0.6056742072105408
    }
  }
}

